In [1]:
import torch
import numpy as np
import wandb

from utils import parse_config, unflatten_dot
from dataset import get_loader
from learner import Learner
from models import ASTPretrainedBigHead, ASTPretrainedSmallHead, RNN, ASTWithWeightedLayerPooling
from types import SimpleNamespace

In [2]:
SEED = 123
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [3]:
def main():
    
    wandb.init()
    config = wandb.config
    train_dl = get_loader(config, subset="train")
    valid_dl = get_loader(config, subset="valid")
    
    model = ASTPretrainedSmallHead(n_classes=11)

    learn = Learner(train_dl, valid_dl, model, config)
    learn.unfreeze()

    learn.fit(4)

In [4]:
import yaml
CONFIG_PATH = "./sweep_config.yaml"
with open(CONFIG_PATH) as file:
    config = yaml.safe_load(file)

sweep_id = wandb.sweep(sweep=config)
wandb.agent(sweep_id, function=main, count=10)

Create sweep with ID: hdafiir7
Sweep URL: https://wandb.ai/k-pintaric/uncategorized/sweeps/hdafiir7


wandb: Agent Starting Run: 7begkphe with config:
wandb: 	EPOCHS: 3
wandb: 	batch_size: 8
wandb: 	early_stopping: {'min_delta': 0.0001, 'patience': 20}
wandb: 	learning_rates: {'base_model': {'lr': 6.281122088848578e-08}, 'classifier': {'lr': 0.00045641324682915507}}
wandb: 	loss: {'FocalLoss': {'alpha': 0.25, 'gamma': 2}}
wandb: 	metrics: ['hamming_score', 'zero_one_score', 'mean_average_precision', 'mean_f1_score']
wandb: 	num_accum: 4
wandb: 	optimizer: {'AdamW': {'weight_decay': 0.008252844099883201}}
wandb: 	preprocess: {'PreprocessPipeline': {'target_sr': 22050}}
wandb: 	save_best_model: True
wandb: 	scheduler: {'OneCycleLR': {'anneal_strategy': 'cos', 'pct_start': 0.14499683328854124}}
wandb: 	signal_augments: None
wandb: 	spec_augments: None
wandb: 	train_dir: ./data/processed_22khz/all_sync/IRMAS_Training_Data
wandb: 	transforms: {'CustomFeatureExtractor': {'hop_length': 256, 'max_length': 1024, 'mean': -4.2677393, 'n_fft': 2048, 'n_mels': 128, 'sample_rate': 22050, 'std': 9.13

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/9220 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

...

| EPOCH: 5 | train_loss: 0.004 | val_loss: 0.046 |

hamming_score: 0.85
zero_one_score: 0.13
mean_average_precision: 0.78
mean_f1_score: 0.60


  0%|          | 0/9220 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

| EPOCH: 6 | train_loss: 0.004 | val_loss: 0.046 |

hamming_score: 0.85
zero_one_score: 0.12
mean_average_precision: 0.78
mean_f1_score: 0.59


epoch,▁▂▄▅▇█
hamming_score,▁▅██▆▅
lr_param_group_0,▁▁▂▂▃▃▄▅▆▇▇███████▇▇▇▇▆▆▅▅▄▄▄▃▃▂▂▂▂▁▁▁▁▁
lr_param_group_1,▁▁▂▂▃▃▄▅▆▇▇███████▇▇▇▇▆▆▅▅▄▄▄▃▃▂▂▂▂▁▁▁▁▁
mean_average_precision,▁▇▆█▆▇
mean_f1_score,▂▄██▄▁
test_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▅▄▃▂▁
train_loss_per_batch,█▇▅▇▆▃▆▅▄▅▄▅▃▂▅▄▂▄▂▄▂▄▁▃▃▂▃▂▃▂▂▃▂▁▁▁▃▁▁▂
train_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▁▂▅▆██
